
<br>
Main file for the SVM_custom project<br>
Authors: Wojciech Trejter, Viki Simion, Laura M Quirós<br>


In [1]:
import ast

In [2]:
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [3]:
def one_hot_encoding(train, test):
    # get all unique strings from the train["emotions"] and test set
    unique_emotions = set(train["emotion"].unique().tolist() + test["emotion"].unique().tolist())
    # one hot encode the emotions
    dict_one_hot = {}
    num = 0
    for emotion in unique_emotions:
        dict_one_hot[emotion] = num
        num += 1
    train["emotion"] = train["emotion"].map(dict_one_hot)
    test["emotion"] = test["emotion"].map(dict_one_hot)
    # save the one hot encoding dictionary
    pd.DataFrame.from_dict(data=dict_one_hot, orient='index').to_csv('data/one_hot_encoding.csv', header=False)
    return train, test

In [4]:
train = pd.read_csv('data/train_ready_for_WS_vectorized_1-gram.csv', delimiter=';', header=0)
test = pd.read_csv('data/test_vectorized_1-gram.csv', delimiter=';', header=0)
train['vector_1-gram'] = train['vector_1-gram'].apply(ast.literal_eval)
test['vector_1-gram'] = test['vector_1-gram'].apply(ast.literal_eval)
train, test = one_hot_encoding(train, test)
train_X = train["vector_1-gram"]
# make the list of values into columns
train_X = pd.DataFrame(train_X.tolist())
train_y = train["emotion"]
test_X = test["vector_1-gram"]
test_X = pd.DataFrame(test_X.tolist())
test_y = test["emotion"]

print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(1860, 100) (1860,) (270, 100) (270,)


In [5]:
params = {
    "kernel": ["linear", "poly", "rbf", "sigmoid"],
    "C": [2, 3, 5, 7, 10]
}
svm = SVC()
grid_search = GridSearchCV(estimator=svm, param_grid=params, error_score='raise', scoring='accuracy')
grid_search.fit(train_X, train_y)
best_model = grid_search.best_estimator_
print(f"Best parameters: {grid_search.best_params_}, best score: {grid_search.best_score_}")
   

Best parameters: {'C': 5, 'kernel': 'rbf'}, best score: 0.3591397849462365


In [6]:
results = classification_report(test_y, best_model.predict(test_X))
print(results)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        76
           1       0.00      0.00      0.00        14
           2       0.12      0.08      0.10        25
           3       0.36      0.93      0.52        98
           4       0.00      0.00      0.00        31
           5       0.00      0.00      0.00        12
           6       0.00      0.00      0.00        14

    accuracy                           0.34       270
   macro avg       0.07      0.14      0.09       270
weighted avg       0.14      0.34      0.20       270


/Users/lauramariaquirosconesa/Documents/BSc/Year3/2ASemester/NLP/NatLanPro/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/lauramariaquirosconesa/Documents/BSc/Year3/2ASemester/NLP/NatLanPro/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/lauramariaquirosconesa/Documents/BSc/Year3/2ASemester/NLP/NatLanPro/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in label